# **First Thing First**

* **Upload Kaggle.json**

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muslikhannur","key":"e1a764bf29bba0afe667930f2a7d397c"}'}

* **Mount Drive**

In [2]:
from google.colab import drive # So the Directory Path will be Visible
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Tweak Datasets Using Kaggle API**

* **Some Installation and Configuration**

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

* **Looking for the Dataset List with Keyword 'COVID'**

In [0]:
!kaggle datasets list -s COVID

* **Force Download and Place to Mounted Drive** - 4 April = 18687 Rows

In [0]:
!kaggle datasets download imdevskp/corona-virus-report -p '/content/drive/My Drive/Colab Notebooks/Projects - COVID19' --unzip -o -q

# **Start The Main Part**

* **Load Libraries**

In [0]:
!pip install pycountry
!pip install pycountry_convert
!pip install pymysql

import pymysql
import pycountry
import pandas as pd
import numpy as np
import pycountry_convert as pc
import datetime

from sqlalchemy import create_engine
from datetime import datetime
from functools import reduce
from datetime import date

* **Load Datasets**

In [7]:
path = '/content/drive/My Drive/Colab Notebooks/Projects - COVID19/'
df_Covid = pd.read_csv(path + 'covid_19_clean_complete.csv')
df_Covid.columns = ['Country_State', 'Country_Name', 'Lat', 'Long',
                    'DateOld', 'Confirmed', 'Deaths', 'Recovered'] 
df_Covid

,Country_State,Country_Name,Lat,Long,DateOld,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
19013,NaN,Botswana,-22.328500,24.684900,4/4/20,4,1,0
19014,NaN,Burundi,-3.373100,29.918900,4/4/20,3,0,0
19015,NaN,Sierra Leone,8.460555,-11.779889,4/4/20,4,0,0
19016,NaN,Malawi,-13.254308,34.301525,4/4/20,4,0,0


# **Pre-Process Date**

* **Append a Column Called Date (datetime64) use to_datetime**

In [8]:
Date = pd.to_datetime(pd.Series(df_Covid['DateOld']), format="%m/%d/%y")
df_Covid['Date'] = Date
df_Covid.dtypes

Country_State            object
Country_Name             object
Lat                     float64
Long                    float64
DateOld                  object
Confirmed                 int64
Deaths                    int64
Recovered                 int64
Date             datetime64[ns]
dtype: object

* **or Append Date (datetime64) using datetime.strptime - Alternative**

In [0]:
# Pull DateOld
getDate = df_Covid['DateOld']

# Function Convert YY-MM-DD from MM/DD/YY (Object)
def object_to_date(DateBefore):
    DateAfter = datetime.strptime(DateBefore, '%m/%d/%y').strftime('%Y-%m-%d')
    return DateAfter

# Append a Column Called Date (New)
Date = []
for i in range(len(getDate)):    
    DateBefore = getDate[i]
    Date.append(object_to_date(DateBefore))
df_Covid['Date'] = Date #But still String
df_Covid.tail() 

# **Pre-Process Country Code**


* **Make a DataFrame for Reference Table, Append Country_Name**

In [9]:
Country_Name = df_Covid['Country_Name'] #Raw
Country_Name = list(set(Country_Name)) #Distinct
Country_Name = sorted(Country_Name) #Sort
df_Country = pd.DataFrame(Country_Name)
df_Country.columns = ['Country_Name']
df_Country.tail()

,Country_Name
176,Venezuela
177,Vietnam
178,West Bank and Gaza
179,Zambia
180,Zimbabwe


* **Append a Column Called Country_Code**

In [10]:
Country_ISO = {} #Read this Variable to Show Official Name
for country in pycountry.countries:
    Country_ISO[country.name] = country.alpha_2
Country_Code = [Country_ISO.get(country) for country in Country_Name] 
df_Country['Country_Code'] = Country_Code
df_Country.tail()

,Country_Name,Country_Code
176,Venezuela,None
177,Vietnam,None
178,West Bank and Gaza,None
179,Zambia,ZM
180,Zimbabwe,ZW


* **Check Missing Value for Country_Code**

In [11]:
df_Country.isnull().values.any() 
df_Country.isnull().sum()

Country_Name     0
Country_Code    22
dtype: int64

* **Check which Country has Missing Country_Code**

In [12]:
Missing = df_Country[df_Country['Country_Code'].isnull()].index.tolist()
Country_Missing_Code = df_Country.loc[Missing, 'Country_Name'] #Save Missing Country
Country_Missing_Code.array

<PandasArray>
[            'Bolivia',              'Brunei',               'Burma',
 'Congo (Brazzaville)',    'Congo (Kinshasa)',       "Cote d'Ivoire",
    'Diamond Princess',            'Holy See',                'Iran',
              'Kosovo',                'Laos',          'MS Zaandam',
             'Moldova',              'Russia',         'South Korea',
               'Syria',             'Taiwan*',            'Tanzania',
                  'US',           'Venezuela',             'Vietnam',
  'West Bank and Gaza']
Length: 22, dtype: object

* **Based on Country_Missing_Code, we must Fill it Manually**

In [13]:
df_Country.loc[Missing, 'Country_Code'] = ['BO', 'BN', 'MM', 'CG', 'CD', 'CI', 'Ship-DP', 
                                            'VA', 'IR', 'XK', 'LA', 'Ship-ZA', 'MD', 'RU', 
                                              'KR', 'SY', 'TW', 'TZ', 'US', 'VE', 'VN', 'PS']                                          
df_Country.tail()

,Country_Name,Country_Code
176,Venezuela,VE
177,Vietnam,VN
178,West Bank and Gaza,PS
179,Zambia,ZM
180,Zimbabwe,ZW


* **Fully Filled, Ready to Transform Processed to the Next Stage**


In [0]:
Country_Code = list(df_Country['Country_Code']) 

# **Pre-Process Continent Name**

* **Function Convert Continent_Name from Country_Code**

In [0]:
def country_to_continent(country_alpha2):
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

* **Append a Column Called Continent_Name**

In [16]:
Continent_Name = []
for i in range(len(Country_Code)):    
    country_alpha2 = Country_Code[i]
    try: Continent_Name.append((country_to_continent(country_alpha2)))
    except: Continent_Name.append(np.NaN)
df_Country['Continent_Name'] = Continent_Name
df_Country.tail()

,Country_Name,Country_Code,Continent_Name
176,Venezuela,VE,South America
177,Vietnam,VN,Asia
178,West Bank and Gaza,PS,Asia
179,Zambia,ZM,Africa
180,Zimbabwe,ZW,Africa


* **Check Missing Value for Continent_Name**





In [17]:
df_Country.isnull().values.any() 
df_Country.isnull().sum() 

Country_Name      0
Country_Code      0
Continent_Name    4
dtype: int64

* **Check which Country has Missing Continent_Name**

In [18]:
Missing = df_Country[df_Country['Continent_Name'].isnull()].index.tolist() 
Country_Missing_Continent = df_Country.loc[Missing, 'Country_Name'] #Save Missing Country
Country_Missing_Continent.array

<PandasArray>
['Diamond Princess', 'Holy See', 'MS Zaandam', 'Timor-Leste']
Length: 4, dtype: object

* **Based on Country_Missing_Continent, we must Fill it Manually**

In [19]:
df_Country.loc[Missing, 'Continent_Name'] = ['Ship', 'Europe', 'Ship', 'Asia']
df_Country.tail()

,Country_Name,Country_Code,Continent_Name
176,Venezuela,VE,South America
177,Vietnam,VN,Asia
178,West Bank and Gaza,PS,Asia
179,Zambia,ZM,Africa
180,Zimbabwe,ZW,Africa


# **Feature Engineering**

* **Merge, Selection, Ordering**

In [20]:
df_Merge = [df_Covid, df_Country] #LEFT JOIN
df_Merge = reduce(lambda left,right: pd.merge(left,right,on='Country_Name',how='left'), df_Merge)

df_Merge = df_Merge.drop(['Lat', 'Long', 'DateOld'], axis = 1) #Unused Features
df_Merge = df_Merge[['Country_State', 'Country_Code', 'Country_Name', 'Continent_Name', 
                     'Date', 'Confirmed', 'Deaths', 'Recovered']] #Sort
                     
df_Merge.dtypes

Country_State             object
Country_Code              object
Country_Name              object
Continent_Name            object
Date              datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
dtype: object

* **Manipulate Index For DB Purpose** 

In [21]:
df_Merge.index += 1 
df_Merge

,Country_State,Country_Code,Country_Name,Continent_Name,Date,Confirmed,Deaths,Recovered
1,NaN,AF,Afghanistan,Asia,2020-01-22,0,0,0
2,NaN,AL,Albania,Europe,2020-01-22,0,0,0
3,NaN,DZ,Algeria,Africa,2020-01-22,0,0,0
4,NaN,AD,Andorra,Europe,2020-01-22,0,0,0
5,NaN,AO,Angola,Africa,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
19014,NaN,BW,Botswana,Africa,2020-04-04,4,1,0
19015,NaN,BI,Burundi,Africa,2020-04-04,3,0,0
19016,NaN,SL,Sierra Leone,Africa,2020-04-04,4,0,0
19017,NaN,MW,Malawi,Africa,2020-04-04,4,0,0


In [22]:
df_Merge.dtypes

Country_State             object
Country_Code              object
Country_Name              object
Continent_Name            object
Date              datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
dtype: object

# **Insert to MySQL DB & BackUp CSV**

* **Creating Connection**

In [0]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{localhost}:{port}/{db}"
                       .format(user="xxx", pw="xxx", localhost="xxx", port = "xxx", db="xxx")) #Change It!

* **Ready to Insert, by Replace**

In [0]:
df_Merge.to_sql('covid_19_clean_complete', con = engine, if_exists = 'replace', chunksize = 1000)

# **BackUp CSV to Local File** 

* **Store CSV At /content**

In [0]:
VersionFile = date.today().strftime("%Y-%m-%d")
df_Merge.to_csv(path + 'covid_19_clean_complete ' + VersionFile + '.csv', index=True) 

* **Download Your CSV, with Versioning**

In [0]:
files.download(path + 'covid_19_clean_complete ' + VersionFile + '.csv')